This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = '5ySUbRtPumxeuYGufWym'

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests
import pandas as pd


In [3]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
url = 'https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?api_key='
complete = url+API_KEY
r = requests.get(complete)
json_data = r.json()

In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
json_dict = dict(json_data)
thelist = []
for i in json_dict.keys():
    print('The key is ' + i + ' and ')
    print(type(json_dict))
    thelist.append(i)

The key is dataset_data and 
<class 'dict'>


In [5]:
secondlist = []
for i in json_dict[i].keys():
    secondlist.append(i)
for i in secondlist:
    print('The "subkey" is '+ i + ' and ')
    print(type(json_dict[thelist[0]][i]))

The "subkey" is limit and 
<class 'NoneType'>
The "subkey" is transform and 
<class 'NoneType'>
The "subkey" is column_index and 
<class 'NoneType'>
The "subkey" is column_names and 
<class 'list'>
The "subkey" is start_date and 
<class 'str'>
The "subkey" is end_date and 
<class 'str'>
The "subkey" is frequency and 
<class 'str'>
The "subkey" is data and 
<class 'list'>
The "subkey" is collapse and 
<class 'NoneType'>
The "subkey" is order and 
<class 'NoneType'>


These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [6]:
df = pd.DataFrame(json_dict['dataset_data']['data'])          # Coerce data with no column names
thedict = {}
for i in df.columns:
    thedict[i] = json_dict['dataset_data']['column_names'][i] # Populate dict specifically for renaming columns
completed_df = df.rename(columns = thedict)                   # Rename column

In [7]:
# Only focus on 2017
completed_df = completed_df[(completed_df['Date'] < '2018-01-01') & (completed_df['Date'] >= '2017-01-01')]

# Build funciton to eliminate any columns that contain single value or 0
def rm_emp_col(data):
    thelist = data.nunique()                                                        # Count each unique
    filtered = pd.DataFrame(thelist[thelist <= 1]).reset_index()['index'].unique()  # Select columns: 1 or 0 unique
    return data.drop(filtered,axis = 1)                                             # Remove these columns

cleaned_df = rm_emp_col(completed_df)
display(cleaned_df.describe())        # 3.) Highest Open: $53.11; Lowest Open: $34.00
                                      # 6.) Mean Trading Volumne: 89,124.34

,Open,High,Low,Close,Traded Volume,Turnover
count,252.000000,255.000000,255.000000,255.000000,255.000000,2.550000e+02
mean,43.344206,43.702804,42.924373,43.364157,89124.337255,3.853589e+06
std,4.348585,4.365667,4.267225,4.321755,60441.130541,2.517807e+06
min,34.000000,34.120000,33.620000,34.060000,45.000000,1.980000e+03
25%,41.395000,41.580000,41.045000,41.400000,56282.000000,2.388636e+06
50%,43.450000,43.550000,42.620000,43.280000,76286.000000,3.292223e+06
75%,45.857500,46.195000,45.390000,45.850000,104479.000000,4.591904e+06
max,53.110000,53.540000,52.480000,53.090000,670349.000000,2.591054e+07


In [8]:
# 4.) What was the largest change in any one day (based on High and Low price)?
cleaned_df['High_Diff'] = (cleaned_df['High'].shift(periods=1) - cleaned_df[   # Use percentage change formula
    'High'])/cleaned_df['High'].shift(periods=1).abs()
cleaned_df['Low_Diff'] = (cleaned_df['Low'].shift(periods=1) - cleaned_df[
    'Low'])/cleaned_df['Low'].shift(periods=1).abs()

display(cleaned_df[cleaned_df['High_Diff'] == cleaned_df['High_Diff'].max()][['Date','High_Diff']])
display(cleaned_df[cleaned_df['Low_Diff'] == cleaned_df['Low_Diff'].max()][['Date','Low_Diff']])

,Date,High_Diff
889,2017-05-10,0.053409


,Date,Low_Diff
753,2017-11-21,0.033271


In [9]:
# 5.) What was the largest change between any two days (based on Closing Price)?
cleaned_df['Close_Diff'] = (cleaned_df['Close'].shift(periods=2) - cleaned_df[   # Use percentage change formula
    'Close'])/cleaned_df['Close'].shift(periods=2).abs()

display(cleaned_df[cleaned_df['Close_Diff'] == cleaned_df['Close_Diff'].max()][['Date','Close_Diff']])

,Date,Close_Diff
953,2017-02-09,0.059175


In [10]:
# 7.) (Optional) What was the median trading volume during this year?
print('The median trading volumen during this year is '+str(cleaned_df['Traded Volume'].median())+'.')

The median trading volumen during this year is 76286.0.
